In [1]:
import numpy as np
import os
from tensorflow import keras
import keras
import cv2 as cv2
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from skimage.measure import regionprops
import matplotlib.patches as mpatches
from skimage.morphology import label
from skimage import *
from scipy import misc, ndimage
from keras.callbacks import TensorBoard
%matplotlib inline

/home/student/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_path = 'Training'
test_path = 'Test'
valid_path = 'Valid'

In [3]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224),classes=['Healthy_Leaves','Unhealthy_Leaves'], batch_size=20)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224),classes=['Healthy_Leaves','Unhealthy_Leaves'], batch_size=20)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224),classes=['Healthy_Leaves','Unhealthy_Leaves'], batch_size=5)

Found 200 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [4]:
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

# Confusion matrix

In [5]:
# test_imgs, test_labels = next(test_batches)
# plots(test_imgs, titles=test_labels)

In [6]:
#rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10,verbose=0)

In [7]:
#cm = confusion_matrix(test_labels, rounded_predictions)

In [8]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label') 
    plt.xlabel('Predicted label')

# Prepoccessing the images

In [9]:
# image_path = 'Training/Unhealthy_Leaves/d2313a95-61dd-45de-a8b8-3c2992e2b97f___JR_FrgE.S 8652.JPG'
# nimg = cv2.imread(image_path,0)
# edges = cv2.Canny(nimg,200,200)

# laplacian = cv2.Laplacian(nimg,cv2.CV_64F)
# sobelx = cv2.Sobel(nimg,cv2.CV_64F,1,0,ksize=5)  # x
# sobely = cv2.Sobel(nimg,cv2.CV_64F,0,1,ksize=5)  # y


# plt.subplot(121),plt.imshow(nimg,cmap = 'gray')
# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(122),plt.imshow(edges,cmap = 'gray')
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(2,2,1),plt.imshow(nimg,cmap = 'gray')
# plt.title('Original'), plt.xticks([]), plt.yticks([])
# plt.subplot(2,2,2),plt.imshow(laplacian,cmap = 'gray')
# plt.title('Laplacian'), plt.xticks([]), plt.yticks([])
# plt.subplot(2,2,3),plt.imshow(sobelx,cmap = 'gray')
# plt.title('Sobel X'), plt.xticks([]), plt.yticks([])
# plt.subplot(2,2,4),plt.imshow(sobely,cmap = 'gray')
# plt.title('Sobel Y'), plt.xticks([]), plt.yticks([])

# plt.show()

In [10]:
def replace_image(image, image_to_be_replaced_path, filename):
    cv2.imwrite(os.path.join(path , filename), image)

In [11]:
# model = Sequential([
#     Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
#     Flatten(),
#     Dense(2,activation='softmax')
# ])
# model.compile(Adam(lr=.0001),loss='categorical_crossentropy',metrics=['accuracy'])
# path = "/home/student/FinalProject/Training/Healthy_Leaves/"
# for i in os.listdir(path):
#     image_path = path + i
#     img = cv2.imread(image_path, 1)#color
# #     cv2.imshow('colored image', img)
# #     cv2.waitKey()
#     nimg = cv2.imread(image_path, 0)#grayscale
#     nimg = cv2.threshold(nimg, 127, 255, cv2.THRESH_BINARY)[1]  # ensure binary
#     image, contours, hier = cv2.findContours(nimg, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#     ret, labels = cv2.connectedComponents(nimg)

#     # Map component labels to hue val
#     label_hue = np.uint8(179*labels/np.max(labels))
#     blank_ch = 255*np.ones_like(label_hue)
#     labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

#     # cvt to BGR for display
#     labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
#     # set bg label to black
#     labeled_img[label_hue==0] = 0
#     w_max = 0
#     h_max = 0
#     for c in contours:
#         x,y,w,h = cv2.boundingRect(c)
#         cv2.rectangle(nimg, (x,y), (x+w, y+h), (0,255,0),2)
#         if (w_max*h_max) < (w*h) and x != 0 and y != 0:
#             x_max = x
#             y_max = y
#             w_max = w
#             h_max = h
#     cv2.rectangle(nimg, (x_max,y_max), (x_max+w_max, y_max+h_max), (0,255,0),2)
#     img = img[y_max:y_max+h_max, x_max:x_max+w_max]
#     replace_image(img, path, i)
# cv2.imshow('labeled.png', labeled_img)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [12]:
# label_image = label(labeled_img)
# blobs = measure.label(label_image, connectivity=1)
# props = measure.regionprops(blobs)
# props[0].bbox
#roi = regionprops(props)
#props[0].area
#cv2.imshow(roi)
# w_max = 0
# h_max = 0
# for c in contours:
#     x,y,w,h = cv2.boundingRect(c)
#     cv2.rectangle(nimg, (x,y), (x+w, y+h), (0,255,0),2)
#     #print(x,y,w,h)
#     if (w_max*h_max) < (w*h) and x != 0 and y != 0:
#         x_max = x
#         y_max = y
#         w_max = w
#         h_max = h

#print(x_max,y_max,h_max,w_max)
# cv2.rectangle(nimg, (x_max,y_max), (x_max+w_max, y_max+h_max), (0,255,0),2)
# cv2.imshow('uncropped image', nimg)
# cv2.waitKey()
# img = img[y_max:y_max+h_max, x_max:x_max+w_max]
# cv2.imshow('cropped image', img)
# cv2.waitKey()
# cv2.destroyAllWindows()

# Training the model

In [13]:
model = Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
    Flatten(),
    Dense(2,activation='softmax')
])

In [14]:
#model.compile(Adam(lr=.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
#model.fit_generator(train_batches, steps_per_epoch=4,validation_data=valid_batches, validation_steps=4,epochs=10,verbose=2)

# Predict

In [16]:
# test_imgs, test_labels = next(test_batches)
# plots(test_imgs, titles=test_labels)

In [17]:
# test_labels = test_labels[:,0]
# test_labels

In [18]:
#predictions = model.predict_generator(test_batches,steps=1,verbose=0)

In [19]:
#predictions

In [20]:
#cm = confusion_matrix(test_labels, predictions[:,0])

In [21]:
#cm_plot_labels = ['Unhealthy', 'Healthy']
#plot_confusion_matrix(cm, cm_plot_labels, title='confusion matrix')

# Fine tuning model

In [22]:
vgg16_model = keras.applications.vgg16.VGG16()

In [23]:
#vgg16_model.summary()

In [24]:
model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

In [25]:
#model.summary()

In [26]:
model.layers.pop()

In [27]:
#model.summary()

In [28]:
for layer in model.layers:
    layer.trainable = False

In [29]:
model.add(Dense(2, activation='softmax'))

In [30]:
#model.summary()

In [31]:
model.compile(Adam(lr=.0001),loss='categorical_crossentropy',metrics=['accuracy'])

fits the model and creates a tensorboard graph

In [32]:
tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
model.fit_generator(train_batches, steps_per_epoch=4,
                   validation_data=valid_batches, validation_steps=4,epochs=5,verbose=2, callbacks=[tensorBoardCallback])

Epoch 1/5
 - 24s - loss: 0.6887 - acc: 0.7875 - val_loss: 0.6828 - val_acc: 0.9500
Epoch 2/5
 - 23s - loss: 0.6789 - acc: 0.9875 - val_loss: 0.6740 - val_acc: 1.0000
Epoch 3/5
 - 23s - loss: 0.6694 - acc: 1.0000 - val_loss: 0.6658 - val_acc: 1.0000
Epoch 4/5
 - 23s - loss: 0.6618 - acc: 1.0000 - val_loss: 0.6587 - val_acc: 1.0000
Epoch 5/5
 - 24s - loss: 0.6570 - acc: 1.0000 - val_loss: 0.6524 - val_acc: 1.0000


In [34]:
# test_imgs, test_labels = next(test_batches)
# plots(test_imgs, titles=test_labels)

In [ ]:
# test_labels = test_labels[:,0]
# test_labels

In [ ]:
# predictions = model.predict_generator(test_batches,steps=1,verbose=0)
# predictions

In [ ]:
# cm = confusion_matrix(test_labels, np.round(predictions[:,0]))
# cm_plot_labels = ['Unhealthy', 'Healthy']
# plot_confusion_matrix(cm, cm_plot_labels, title='confusion matrix')

In [38]:
image = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224),classes=['Leaves'], batch_size=20)
classify_image = model.predict_generator(image,steps=1,verbose=0)
classify_image

Found 1 images belonging to 1 classes.


array([[0.5144205, 0.4855795]], dtype=float32)

# Data Augmentation

In [ ]:
gen = ImageDataGenerator(rotation_range=10,width_shift_range=0.1,
                        height_shift_range=0.1,shear_range=0.15,zoom_range=0.1,
                        channel_shift_range=10.,horizontal_flip=True)


In [ ]:
image_path = '/home/student/FinalProject/Training/Healthy_Leaves/Healthy_Leaf_1.jpg'

In [ ]:
#obtain image
image = np.expand_dims(ndimage.imread(image_path),0)
plt.imshow(image[0])

In [ ]:
aug_iter = gen.flow(image)

In [ ]:
aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(10)]

In [ ]:
plots(aug_images,figsize=(20,7), rows=2)

In [ ]:
test_imgs, test_labels = next(aug_images)
plots(aug_images, titles=test_labels)
test_labels = test_labels[:,0]
test_labels

In [ ]:
predictions = model.predict_generator(test_batches,steps=1,verbose=0)
predictions

In [ ]:
cm = confusion_matrix(test_labels, np.round(predictions[:,0]))
cm_plot_labels = ['Unhealthy', 'Healthy']
plot_confusion_matrix(cm, cm_plot_labels, title='confusion matrix')

# Bias

In [ ]:
model = Sequential([
    Dense(4,input_shape=(1,),activation='relu',use_bias=True,bias_initializer='zeros'),
    Dense(2,activation='softmax')
])

In [ ]:
model.get_weights()

Plan for finshing up this project:
I need to preproccess every image instead of just the one
load the images into the model
using data augmentation and bias and maybe more to complete the model
then predict a test img and see the accuarcy
After all of that, try to make it so you can input any image you want
then maybe try to turn it into a andriod app.(this will take a while)